#피지엠파이를 사용한 베이즈 정리 적용¶

피지엠파이 패키지는 베이즈 정리에 적용하는 BayesianModel 클래스를 제공한다. 베이즈 정리를 적용하려면 조건부확률을 구현하는 TabularCPD 클래스를 사용하여 사전확률과 가능도를 구현해야 한다. TabularCPD 클래스 객체는 다음과 같이 만든다.

TabularCPD(variable, variable_card, value, evidence=None, evidence_card=None)

variable: 확률변수의 이름 문자열

variable_card: 확률변수가 가질 수 있는 경우의 수

value: 조건부확률 배열. 하나의 열(column)이 동일 조건을 뜻하므로 하나의 열의 확률 합은 1이어야 한다.

evidence: 조건이 되는 확률변수의 이름 문자열의 리스트

evidence_card: 조건이 되는 확률변수가 가질 수 있는 경우의 수의 리스트

TabularCPD 클래스는 원래는 조건부확률을 구현하기 위한 것이지만 evidence=None, evidence_card=None으로 인수를 주면 일반적인 확률도 구현할 수 있다.

### 예시 :  검사 시약 문제

제약사에서 환자가 특정한 병에 걸린지 확인하는 시약을 만들었다. 그 병에 걸린 환자에게 시약을 테스트한 결과 99%의 확률로 양성 반응을 보였다. 병에 걸린지 확인이 되지 않은 어떤 환자가 이 시약을 테스트한 결과 양성 반응을 보였다면 이 환자가 그 병에 걸려 있을 확률은 얼마인가? 99%일까?

- 병에 걸린 경우 : 사건 $D$
- 병에 걸리지 않은 경우 : 사건 $D^c$
- 양성반응을 보이는 경우 : 사건 $S$
- 병에 걸린 사람이 양성인 경우 : 조건부 사건 $S|D$
- 양성 반응을 보이는 사람이 병에 걸려 있을 경우 : 조건부 사건 $D|S$
- $P(S|D) = 0.99$ 일 때, $P(D|S)$를 구해라
- 이 병은 전체 인구 중 걸린 사람이 0.2%인 희귀병 : $P(D) = 0.002$
- 이 병에 걸리지 않은 사람에게 시약 검사를 했을 때, 양성 반응, 즉 잘못된 결과(False Positive)가 나타난 확률은 5%이다. : $P(S|D^c)=0.05$

우선 확률변수 $X$를 이용하여 병에 걸렸을 사전확률 $P(D)=P(X=1)$, 병에 걸리지 않았을 사전확률 $P(D^C)=P(X=0)$를 정의한다.

In [8]:
from pgmpy.factors.discrete import TabularCPD
import numpy as np

cpd_X = TabularCPD('X', 2, [[1 - 0.002],[0.002]])
print(cpd_X)

+------+-------+
| X(0) | 0.998 |
+------+-------+
| X(1) | 0.002 |
+------+-------+


다음으로는 양성 반응이 나올 확률 $P(S)=P(Y=1)$, 음성 반응이 나올 확률 $P(S^C)=P(Y=0)$를 나타내는 확률변수 $Y$를 정의한다.

확률변수 $Y$의 확률을 베이즈 모형에 넣을 때는 TabularCPD 클래스를 사용한 조건부확률 $P(Y|X)$의 형태로 넣어야 하므로 다음처럼 조건부확률 $P(Y|X)$를 구현한다.

In [9]:
cpd_Y_on_X = TabularCPD('Y', 2, np.array([[0.95, 0.01], [0.05, 0.99]]),
                        evidence=['X'], evidence_card=[2])
print(cpd_Y_on_X)

+------+------+------+
| X    | X(0) | X(1) |
+------+------+------+
| Y(0) | 0.95 | 0.01 |
+------+------+------+
| Y(1) | 0.05 | 0.99 |
+------+------+------+


이제 이 확률변수들이 어떻게 결합되어 있는지는 나타내는 확률모형인 BayesianModel 클래스 객체를 만들어야 한다.

BayesianModel(variables)
- variables: 확률모형이 포함하는 확률변수 이름 문자열의 리스트

BayesianModel 클래스는 다음 메서드를 지원한다.

- add_cpds(): 조건부확률을 추가

- check_model(): 모형이 정상적인지 확인. True면 정상적인 모형

In [11]:
from pgmpy.models import BayesianModel

model = BayesianModel([('X', 'Y')])
model.add_cpds(cpd_X, cpd_Y_on_X)
model.check_model()

C:\Users\user\anaconda3\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


True

BayesianModel 클래스는 변수 제거법(VariableElimination)을 사용한 추정을 제공한다. VariableElimination 클래스로 추정(inference) 객체를 만들고 이 객체의 query() 메서드를 사용하면 사후확률을 계산한다.

query(variables, evidences)
- variables: 사후확률을 계산할 확률변수의 이름 리스트

- evidences: 조건이 되는 확률변수의 값을 나타내는 딕셔너리

여기에서는 pgmpy 패키지를 이용하여 베이즈 정리를 적용할 수 있다는 것만 알면 된다. 자세한 내용은 추후 **확률적 그래프 모형(Probabilistic Graphical Model)**에서 다룬다.

In [12]:
from pgmpy.inference import VariableElimination

inference = VariableElimination(model)
posterior = inference.query(['X'], evidence={'Y': 1}, joint=False, show_progress=False)
print(posterior['X'])

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(0) |   0.9618 |
+------+----------+
| X(1) |   0.0382 |
+------+----------+
